imoprt libraries and make sure we can read datetime

In [1]:
import pandas as pd
from datetime import datetime

copy file path to be able to open file from computer path

In [3]:
#Load the CSV file
file_path = "/content/sps.csv" # Update this to the path of your CSV file
data = pd.read_csv(file_path)

In [4]:
print(data['systolic_peak_timestamp'].dtype)
data['systolic_peak_timestamp'] = pd.to_numeric(data['systolic_peak_timestamp'], errors='coerce')
data = data.dropna(subset=['systolic_peak_timestamp'])


int64


In [5]:
# Convert Unix timestamps from nanoseconds to human-readable datetime
data['readable_datetime'] = data['systolic_peak_timestamp'].apply(
    lambda x: datetime.utcfromtimestamp(x / 1e9).strftime('%Y-%m-%d %H:%M:%S.%f')
)


In [6]:
# Save the processed file
output_path = "systolic_peak_processed.csv"  # Update this to your desired output path
data.to_csv(output_path, index=False)

In [7]:
# Preview the first few rows
print(data.head())

   systolic_peak_timestamp           readable_datetime
0      1706314452436737143  2024-01-27 00:14:12.436737
1      1706314453092975670  2024-01-27 00:14:13.092976
2      1706314453721613746  2024-01-27 00:14:13.721614
3      1706314454259211789  2024-01-27 00:14:14.259212
4      1706314454986003560  2024-01-27 00:14:14.986003


In [8]:
# Create a new column for minute-level grouping
data['minute'] = data['systolic_peak_timestamp'].apply(
    lambda x: datetime.utcfromtimestamp(x / 1e9).strftime('%Y-%m-%d %H:%M')
)


In [9]:
# Group by 'minute' and count the entries
minute_counts = data.groupby('minute').size().reset_index(name='entry_count')

In [10]:
# Merge the counts back into the original data for reference
data = data.merge(minute_counts, on='minute', how='left')

# Save the result to a new file
output_path = "systolic_peak_per_minute.csv"  # Update this to your desired output path
data.to_csv(output_path, index=False)

In [11]:
print(minute_counts.head())

             minute  entry_count
0  2024-01-27 00:14           80
1  2024-01-27 00:15           89
2  2024-01-27 00:16          104
3  2024-01-27 00:17           73
4  2024-01-27 00:18           93


### read data, convert it into timestamp, it is a 24 hour

In [12]:
# Save a simplified DataFrame with only 'minute' and 'entry_count' columns
minute_counts.to_csv("systolic_peak_minute_counts.csv", index=False)